In [2]:
import matplotlib.pyplot as plt
import numpy.ma as ma
import numpy as np
import pandas as pd
import netCDF4
from glob import glob
files = pd.Series(sorted(glob("../pvol/outputs/**/schout_*.nc", recursive=True)))
files

0        ../pvol/outputs/1994/02/schout_1.nc
1       ../pvol/outputs/1994/02/schout_10.nc
2       ../pvol/outputs/1994/02/schout_11.nc
3       ../pvol/outputs/1994/02/schout_12.nc
4       ../pvol/outputs/1994/02/schout_13.nc
                        ...                 
9826     ../pvol/outputs/2020/12/schout_5.nc
9827     ../pvol/outputs/2020/12/schout_6.nc
9828     ../pvol/outputs/2020/12/schout_7.nc
9829     ../pvol/outputs/2020/12/schout_8.nc
9830     ../pvol/outputs/2020/12/schout_9.nc
Length: 9831, dtype: object

In [3]:
nc = netCDF4.Dataset(files[0])
nc

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4_CLASSIC data model, file format HDF5):
    Conventions: CF-1.0, UGRID-1.0
    title: SCHISM Model output
    institution: SCHISM Model output
    source: SCHISM model output version v10
    references: http://ccrm.vims.edu/schismweb/
    history: created by combine_output11
    comment: SCHISM Model output
    type: SCHISM Model output
    VisIT_plugin: https://schism.water.ca.gov/library/-/document_library/view/3476283
    dimensions(sizes): nSCHISM_hgrid_node(126942), nSCHISM_hgrid_face(225109), nSCHISM_hgrid_edge(352104), nMaxSCHISM_hgrid_face_nodes(4), nSCHISM_vgrid_layers(33), one(1), two(2), sigma(33), time(24)
    variables(dimensions): float64 time(time), int32 SCHISM_hgrid(one), int32 SCHISM_hgrid_face_nodes(nSCHISM_hgrid_face, nMaxSCHISM_hgrid_face_nodes), int32 SCHISM_hgrid_edge_nodes(nSCHISM_hgrid_edge, two), float64 SCHISM_hgrid_node_x(nSCHISM_hgrid_node), float64 SCHISM_hgrid_node_y(nSCHISM_hgrid_node), int32 node_bot

In [39]:
safe_dict = {}
for key, value in nc.variables.items():
  safe_subdict = {}
  for k, v in value.__dict__.items():
    if type(v) == np.int32:
      v = int(v)
    elif type(v) == np.float32:
      v = float(v)
    safe_subdict[k] = v
  safe_dict[key] = safe_subdict
safe_dict

{'time': {'long_name': 'Time',
  'units': 'seconds since 1994-02-01 00:00:00 +1200',
  'base_date': ' 1994  2  1       0.00     -12.00',
  'standard_name': 'time'},
 'SCHISM_hgrid': {'long_name': 'Topology data of 2d unstructured mesh',
  'topology_dimension': 2,
  'cf_role': 'mesh_topology',
  'node_coordinates': 'SCHISM_hgrid_node_x SCHISM_hgrid_node_y',
  'face_node_connectivity': 'SCHISM_hgrid_face_nodes',
  'edge_coordinates': 'SCHISM_hgrid_edge_x SCHISM_hgrid_edge_y',
  'face_coordinates': 'SCHISM_hgrid_face_x SCHISM_hgrid_face_y',
  'edge_node_connectivity': 'SCHISM_hgrid_edge_nodes'},
 'SCHISM_hgrid_face_nodes': {'long_name': 'Horizontal Element Table',
  'cf_role': 'face_node_connectivity',
  'start_index': 1,
  '_FillValue': -99999},
 'SCHISM_hgrid_edge_nodes': {'long_name': 'Map every edge to the two nodes that it connects',
  'cf_role': 'edge_node_connectivity',
  'start_index': 1},
 'SCHISM_hgrid_node_x': {'long_name': 'node x-coordinate',
  'standard_name': 'longitude',
 

In [5]:
nc.variables["depth"].shape

(126942,)

In [6]:
lng = nc["SCHISM_hgrid_node_x"][:]
lat = nc["SCHISM_hgrid_node_y"][:]
depth = nc["depth"][:]
lng.shape, lat.shape, depth.shape

((126942,), (126942,), (126942,))

In [7]:
nc["temp"][0, :, :].shape

(126942, 33)

In [8]:
nc["temp"]

<class 'netCDF4._netCDF4.Variable'>
float32 temp(time, nSCHISM_hgrid_node, nSCHISM_vgrid_layers)
    missing_value: 9.96921e+36
    mesh: SCHISM_hgrid
    data_horizontal_center: node
    data_vertical_center: full
    i23d: 2
    ivs: 1
unlimited dimensions: time
current shape = (24, 126942, 33)
filling on, default _FillValue of 9.969209968386869e+36 used

In [9]:
nc["time"]

<class 'netCDF4._netCDF4.Variable'>
float64 time(time)
    long_name: Time
    units: seconds since 1994-02-01 00:00:00 +1200
    base_date:  1994  2  1       0.00     -12.00
    standard_name: time
unlimited dimensions: time
current shape = (24,)
filling on, default _FillValue of 9.969209968386869e+36 used

In [10]:
nc["time"][:] / 60 / 60

masked_array(data=[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0,
                   11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0,
                   20.0, 21.0, 22.0, 23.0, 24.0],
             mask=[False, False, False, False, False, False, False, False,
                   False, False, False, False, False, False, False, False,
                   False, False, False, False, False, False, False, False],
       fill_value=1e+20)

In [11]:
timestamps = pd.to_datetime(netCDF4.num2date(nc["time"], nc["time"].getncattr("units")).astype(str), utc=True).tz_convert("Pacific/Auckland")
timestamps

DatetimeIndex(['1994-02-01 02:00:00+13:00', '1994-02-01 03:00:00+13:00',
               '1994-02-01 04:00:00+13:00', '1994-02-01 05:00:00+13:00',
               '1994-02-01 06:00:00+13:00', '1994-02-01 07:00:00+13:00',
               '1994-02-01 08:00:00+13:00', '1994-02-01 09:00:00+13:00',
               '1994-02-01 10:00:00+13:00', '1994-02-01 11:00:00+13:00',
               '1994-02-01 12:00:00+13:00', '1994-02-01 13:00:00+13:00',
               '1994-02-01 14:00:00+13:00', '1994-02-01 15:00:00+13:00',
               '1994-02-01 16:00:00+13:00', '1994-02-01 17:00:00+13:00',
               '1994-02-01 18:00:00+13:00', '1994-02-01 19:00:00+13:00',
               '1994-02-01 20:00:00+13:00', '1994-02-01 21:00:00+13:00',
               '1994-02-01 22:00:00+13:00', '1994-02-01 23:00:00+13:00',
               '1994-02-02 00:00:00+13:00', '1994-02-02 01:00:00+13:00'],
              dtype='datetime64[ns, Pacific/Auckland]', freq=None)

In [12]:
df = pd.DataFrame({"lat": lat, "lng": lng, "depth": depth})
df

,lat,lng,depth
0,-36.792179,174.591476,-1.813053
1,-36.791809,174.591491,-1.194785
2,-36.792417,174.591583,-3.265060
3,-36.791022,174.591622,-2.264964
4,-36.791433,174.591738,-1.222115
...,...,...,...
126937,-36.479268,176.049355,209.228394
126938,-36.597837,176.065797,158.283493
126939,-36.520800,176.070922,204.225693
126940,-36.552884,176.087601,205.571198


In [65]:
hour = 1
values = nc["temp"][hour, :, :]
values.shape

(126942, 33)

In [66]:
lng = nc["SCHISM_hgrid_node_x"][:]
lat = nc["SCHISM_hgrid_node_y"][:]

In [67]:
variable = "temp"

In [70]:
dfs = []
for depth_level in range(values.shape[-1]):
    values_at_depth = values[:, depth_level]
    df = pd.DataFrame({"lat": lat, "lng": lng, variable: values_at_depth, "depth": depth_level})
    dfs.append(df)
df = pd.concat(dfs)
df.dropna()

,lat,lng,temp,depth
74890,-35.662912,175.059429,11.232645,0
103485,-35.719959,175.380444,11.480674,0
114386,-35.740046,175.429905,11.455647,0
120164,-35.757253,175.460866,11.490908,0
123977,-35.774452,175.491840,11.500798,0
...,...,...,...,...
126937,-36.479268,176.049355,19.455965,32
126938,-36.597837,176.065797,19.440374,32
126939,-36.520800,176.070922,19.458969,32
126940,-36.552884,176.087601,19.448130,32
